## Prep


In [1]:
import openai
from dotenv import load_dotenv
import os
from langfuse import Langfuse
from langchain_community.document_loaders import PyPDFLoader  
import os
import re
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph
from langfuse import Langfuse
from langfuse.callback import CallbackHandler
from langchain.schema import Document
from datetime import datetime
from datetime import datetime

# from langfuse.client import CreateTracer, Creategeneration, CreateSpan


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

client = openai.OpenAI(api_key=OPENAI_API_KEY)
langfuse = Langfuse( 
    secret_key=LANGFUSE_SECRET_KEY,
    public_key=LANGFUSE_PUBLIC_KEY,
    host="https://us.cloud.langfuse.com",
)

c:\Users\yjw64\projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## PDF Read Agent + RAG using Pinecone


### PDF Read Agent


In [2]:
import os
import openai
from pdf2image import convert_from_path
from PIL import Image
from io import BytesIO
import base64

In [3]:
# ✅ OpenAI API 키 설정
openai.api_key = OPENAI_API_KEY

# ✅ PDF → 이미지 변환
def convert_pdf_to_images(pdf_path, dpi=150, poppler_path=None):
    return convert_from_path(pdf_path, dpi=dpi, poppler_path=poppler_path)

# ✅ 이미지 → base64 변환
def encode_image_to_base64(image: Image.Image) -> str:
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

# ✅ Vision API 호출
def summarize_image(image: Image.Image, prompt="이 이미지를 요약해줘"):
    base64_image = encode_image_to_base64(image)
    response = openai.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {
                    "url": f"data:image/png;base64,{base64_image}" }}
            ]}
        ],
        max_tokens=1000
    )
    return response.choices[0].message.content

def ask_gpt4_vision(base64_image: str, prompt: str) -> str:
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
            ]
        }],
        max_tokens=1000
    )
    return response.choices[0].message.content

In [4]:
pdf_paths = [r"C:\Users\yjw64\projects\github\kairos\KAIROS_Podcast\example_files\[Lecture] 12. IP in Linux.pdf" ] # 여기에 PDF 경로 입력
documents = []
date = datetime.today().strftime("%Y-%m-%d") 

# PDF 파일에서 각 페이지를 이미지로 변환하고 요약 생성
for path in pdf_paths:
    if not os.path.exists(path):
        print(f"파일 없음: {path}")
        continue
    lecture_name = os.path.basename(path)
    images = convert_pdf_to_images(path, dpi=150, poppler_path=r"C:\Users\yjw64\projects\github\kairos\KAIROS_Podcast\poppler-24.08.0\Library\bin")
    for i, image in enumerate(images):
        print(f"Processing page {i + 1} of {lecture_name}")
        base64_image = encode_image_to_base64(image)
        summary = ask_gpt4_vision(base64_image, prompt= "이 페이지의 주요 내용을 요약해줘")
        documents.append(Document(
            page_content=summary,
            metadata={
                "lecture_name": lecture_name,
                "upload_date": date,
                "page": i + 1
            }
        ))
        print(f"Page {i + 1} summary done\n")


Processing page 1 of [Lecture] 12. IP in Linux.pdf
Page 1 summary done

Processing page 2 of [Lecture] 12. IP in Linux.pdf
Page 2 summary done

Processing page 3 of [Lecture] 12. IP in Linux.pdf
Page 3 summary done

Processing page 4 of [Lecture] 12. IP in Linux.pdf
Page 4 summary done

Processing page 5 of [Lecture] 12. IP in Linux.pdf
Page 5 summary done

Processing page 6 of [Lecture] 12. IP in Linux.pdf
Page 6 summary done

Processing page 7 of [Lecture] 12. IP in Linux.pdf
Page 7 summary done

Processing page 8 of [Lecture] 12. IP in Linux.pdf
Page 8 summary done

Processing page 9 of [Lecture] 12. IP in Linux.pdf
Page 9 summary done

Processing page 10 of [Lecture] 12. IP in Linux.pdf
Page 10 summary done

Processing page 11 of [Lecture] 12. IP in Linux.pdf
Page 11 summary done

Processing page 12 of [Lecture] 12. IP in Linux.pdf
Page 12 summary done

Processing page 13 of [Lecture] 12. IP in Linux.pdf
Page 13 summary done

Processing page 14 of [Lecture] 12. IP in Linux.pdf
Page

In [5]:
documents[1]

Document(metadata={'lecture_name': '[Lecture] 12. IP in Linux.pdf', 'upload_date': '2025-05-27', 'page': 2}, page_content='이 페이지는 IP(인터넷 프로토콜)에 대한 내용을 다룹니다. 주요 내용은 다음과 같습니다:\n\n1. **IP 개요**\n   - IP 리뷰\n   - IP의 데이터 구조\n\n2. **IP 주요 기능: 라우팅**\n   - IP 라우팅 절차\n   - IP 라우팅을 위한 데이터 구조\n\n3. **IP의 실제 구현**\n   - IP 구현 아키텍처\n   - IP 계층으로의 패킷 소스\n   - IP 출력\n   - IP 입력\n   - 패킷 전달\n\n이 내용은 시스템 프로그래밍 수업의 일환으로 제시되었습니다.')

### Save to vectorstore


In [6]:
# OpenAIEmbeddings 인스턴스 생성
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model="text-embedding-3-small"
)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("kairos-podcast")

C:\Users\yjw64\AppData\Local\Temp\ipykernel_16196\1909058095.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


In [7]:
#check connection
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'userqna': {'vector_count': 6}},
 'total_vector_count': 6,
 'vector_type': 'dense'}


In [8]:
#저장할 vector store 불러오기
vector_store = PineconeVectorStore(
    index=index, 
    embedding=embeddings,
    namespace="lecturedata"
)

In [9]:
#split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

In [10]:
vector_store.add_documents(
    documents=split_docs
)

['d5b6ee96-90d2-4cad-b115-37c5761203d6',
 'f7b08ca9-005a-4def-8375-352dbbeb83d0',
 'f8a131f8-00b6-472b-9366-6ef9bfc49c48',
 'cf5f4026-49ef-4f0a-ba00-ec6ac2506724',
 '59826e76-a347-41c8-bbad-fc8fb27607ce',
 '9302a03c-2cc6-4c9a-ac95-f71d6d37b63f',
 '6208053d-b1ad-4fb7-8fca-a6758ef39fa5',
 'c3413211-8723-496f-a2e3-711d9b8332d1',
 '4e102bd2-90fd-44ab-a6d6-65f5438e346f',
 '2c2e9c85-2c6c-4570-b925-fc325e03f0a5',
 '46eb1f1f-791f-4716-b543-e0df8546272d',
 '77083851-07b4-4ea8-8a23-9514eb95a17d',
 '9c3f9a99-31ab-4722-931a-424ad0cefcb7',
 '13fcaa2e-776a-4c98-b556-43104fec14db',
 '64516afe-9af0-4fee-a041-cd30c24317b1',
 'fb98f229-4f27-429a-a019-218c9a1251c9',
 '6ea450e4-02ac-4259-92ec-d2ab5d0e8bf8',
 '51444c37-a113-4d8b-9567-88eb95e3deda',
 '2c83a6f9-5157-44c7-a46a-659263d321bc',
 'c2a700e0-41bf-4db0-9060-9e70cb3e1508',
 'cd3151bb-6f1e-4b6d-847b-cae291f08329',
 'bb5bb8c8-c499-433a-9e59-5aed5a7ec92b',
 '04d7c92a-8180-4023-ae3b-b6b5d796521e',
 '592d7cba-b4b8-4c39-a27d-febbd44b8fac',
 'cd7f377c-7668-

In [92]:
# # retrieve documents Agent

# def retrieve_docs(state):
#     query= state["init"]
#     lecture_docs = vectorstore.similarity_search(query, k=5, namespace="lecture")
#     qa_docs = vectorstore.similarity_search(query, k=3, namespace="userqa")
#     all_docs = lecture_docs + qa_docs
#     return {"docs": all_docs, "query": query}

## Summary Agent (Generate Script)


In [11]:
def read_prompt(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return f.read().strip()

#if not related QA found, return empty list
def get_qa_docs_with_fallback(query, k):
    try:
        docs = vector_store.similarity_search(query, k=k, namespace="userqna")
        if docs:
            return docs
        else:
            # Fallback: If no documents found, return a default message
            return []
    except Exception as e:
        print("에러 발생:", e)
        return []
    

#generate script Agent

def generate_script(state):

    # 1. Retrieve documents
    query= state["init"]
    lecture_docs = vector_store.similarity_search(query, k=5, namespace="lecture")
    qa_docs = get_qa_docs_with_fallback(query, k=1)
    all_docs = lecture_docs + qa_docs

    # 2. Create user and system prompts
    content = "\n\n".join([doc.page_content for doc in all_docs])
    user_prompt = f"다음은 참고할 문서 내용입니다:\n\n{content}"
    system_prompt = read_prompt("summary_prompt.txt")


    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7
    )
    return {"script": response.choices[0].message.content, "docs": all_docs, "query": query}


## User QnA Agent (Added to Vector Store)


In [12]:
def answer_from_vectorstore(state):
    user_query = state["user_query"]
    docs = vector_store.similarity_search(user_query, k=5, namespace="lecturedata")
    content = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"다음 문서를 참고하여 사용자의 질문에 답해주세요:\n\n{content}\n\n질문: {user_query}"
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )
    answer = response.choices[0].message.content

    # vectorstore에 질문/답변 저장
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    qa_doc = Document(
        page_content=f"Q: {user_query}\nA: {answer}",
        metadata={"source": "userqa", "timestamp": now}
    )
    vector_store.add_documents([qa_doc], namespace="userqna")
    return {"answer": answer} 

## TTS Agent


In [13]:
# generate TTS Agent

def synthesize_tts(state):
    script = state["script"]
    audio_response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=script
    )
    file_name = "podcast_script.mp3"
    with open(file_name, "wb") as f:
        f.write(audio_response.content)
    return {"script": script, "audio_file": file_name}

## MultiAgent


In [14]:
from langfuse import Langfuse
from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler(
  secret_key=LANGFUSE_SECRET_KEY,
  public_key=LANGFUSE_PUBLIC_KEY,
  host="https://us.cloud.langfuse.com"
)

In [15]:
#LangGraph State definition
from typing import TypedDict, Optional


class ChatState(TypedDict):
    init: str
    docs: Optional[list[Document]]
    script: Optional[str]
    audio_file: Optional[str]
    user_query: Optional[str]
    answer: Optional[str]

In [ ]:
#agent node definition

def script_generation(state: ChatState) -> ChatState:
    generated_script = generate_script(state)
    return {**state, "script": generated_script["script"]}

def tts_generation(state: ChatState) -> ChatState:
    tts_result = synthesize_tts(state)
    return {**state, "audio_file": tts_result["audio_file"]}

def save_to_vector_store(state: ChatState) -> None:
    user_qna_data = answer_from_vectorstore(state)
    return {**state, "answer": user_qna_data["answer"]}
    

In [101]:
#workflow connection
from langchain_core.runnables import RunnableLambda

workflow = StateGraph(ChatState)

workflow.add_node("Script Generation", RunnableLambda(script_generation))
workflow.add_node("TTS Generation", RunnableLambda(tts_generation))
workflow.add_node("User QnA", RunnableLambda(save_to_vector_store))

workflow.set_entry_point("Script Generation")
workflow.add_edge("Script Generation", "TTS Generation")
workflow.add_edge("TTS Generation", "User QnA")
workflow.set_finish_point("User QnA")

graph = workflow.compile().with_config({"callbacks": [langfuse_handler]})

In [110]:
state = {
    "init": "강의 내용을 요약한 팟캐스트를 만들어줘",
    "user_query" : "ip_finish_output 과 ip_finish_output2의 차이점은?",
    "docs": None,
    "script": None,
    "audio_file": None,
    "answer": None
}


result = graph.invoke(state)

print(result)

{'init': '강의 내용을 요약한 팟캐스트를 만들어줘', 'docs': None, 'script': '[인트로 음악]\n\n안녕하세요, 여러분! \'테크 토크\'에 오신 걸 환영합니다! 저는 여러분의 팟캐스트 호스트, 카이로스입니다. 오늘은 조금은 기술적인 이야기를 해보려고 해요. 바로 소프트웨어 개발에서 자주 언급되는 함수, `ip_finish_output`과 `ip_finish_output2`에 대한 이야기입니다.\n\n자, 여러분. 혹시 프로그래밍을 하면서 비슷한 이름의 함수들을 보고 "이게 뭐가 다른 거지?" 하고 헷갈렸던 적 있지 않나요? 오늘 다룰 주제가 바로 그런 케이스입니다. `ip_finish_output`과 `ip_finish_output2`, 이름만 보면 둘 다 무언가를 \'끝내는\' 역할을 하는 것 같죠? 하지만 구체적으로 어떻게 다르고, 왜 이렇게 이름이 비슷한 두 함수가 존재하는지 궁금하지 않으세요?\n\n아쉽게도, 제가 직접 그 차이점을 설명하는 문서를 가지고 있진 않아요. 그래서 오늘은 여러분이 이런 상황에 마주쳤을 때, 어떻게 접근하면 좋을지에 대해 이야기해보려고 합니다. \n\n먼저, 함수의 차이를 이해하려면 그 함수들이 정의된 문서나 소스 코드를 직접 살펴보는 게 가장 중요해요. 각 함수의 목적이 무엇인지, 어떤 매개변수를 받는지, 반환값은 무엇인지, 그리고 내부적으로 어떤 로직으로 구현되어 있는지를 살펴보는 것이죠.\n\n예를 들어, `ip_finish_output`과 `ip_finish_output2`라는 함수가 있다고 칩시다. 이 두 함수는 아마도 비슷한 기능을 수행하면서 약간의 차이점을 가질 가능성이 큽니다. 이런 경우, 함수의 이름만 보고 판단하기보다는, 문서화된 설명이나 주석을 참고하여 두 함수가 정확히 어떤 기능을 수행하는지 비교해보는 것이 중요해요.\n\n또한, 이 함수들이 사용되는 실제 사례를 살펴보는 것도 좋은 방법입니다. 소스 코드에서 이 함수들이 어떻게 사용되는지를 보면, 일반적으로 예상되는 사용 패턴

In [103]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	Script_Generation(Script Generation)
	TTS_Generation(TTS Generation)
	User_QnA(User QnA)
	__end__([<p>__end__</p>]):::last
	Script_Generation --> TTS_Generation;
	TTS_Generation --> User_QnA;
	__start__ --> Script_Generation;
	User_QnA --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

